In [ ]:
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
from skimage.filters.rank import entropy as sk_entropy
from skimage.morphology import disk

### Setup variables

In [ ]:
 # VARIABLES TO CHANGE FOR YOUR LOCAL ENVIRONMENT
base_dir = "/home/swhite/vbox_share/cell_proliferation_elliott/ground_truth"
apt_gt_img_dir = os.path.join(base_dir, "extracted_regions")
apt_gt_mask_dir = os.path.join(base_dir, "extracted_regions/gt_masks")

In [ ]:
min_kern_size = (3, 3)

# read in the apartment reference mask
# NOTE: we are eroding the ref mask by a number of iterations here to make the external mask more aggressive
ref_erode_iter = 5
apt_ref_path = 'apt_ref_v3.tif'
apt_ref_mask = cv2.imread(apt_ref_path, cv2.IMREAD_GRAYSCALE)
apt_ref_mask = cv2.erode(apt_ref_mask, np.ones(min_kern_size), iterations=ref_erode_iter).astype(np.bool)

apt_shape = apt_ref_mask.shape

In [ ]:
# Make the backgound mask from the apt image shape
# The background mask is an 11 pixel radius disk, 
# placed here to avoid any cells in the ground truth
# data set.
#
# WARNING: DO NOT USE THIS BACKGROUND SAMPLE LOCATION IN PRODUCTION
#     In the real library, it should be outside the apartment 
#     because there is no guarantee any region within the 
#     apartment is free from cells or debris.
bkg_mask = np.zeros(apt_shape)
bkg_mask = cv2.circle(bkg_mask, (130, 290), 11, 1, -1)
bkg_mask = bkg_mask.astype(np.bool)

### Load ground truth data

In [ ]:
gt_imgs = sorted(glob.glob(os.path.join(apt_gt_img_dir, '*.tif')))
gt_masks = sorted(glob.glob(os.path.join(apt_gt_mask_dir, '*gt_mask.tif')))

In [ ]:
print(len(gt_imgs), len(gt_masks))

In [ ]:
# Organize ground truth data as a dictionary where the main key
# is the apartment ID, value is a dictionary with the 'img' and 'mask'
# keys having the corresponding file paths as values
gt_data = {}

for i, img_path in enumerate(gt_imgs):
    apt_id = os.path.basename(img_path)[-11:-4]
    
    gt_data[apt_id] = {
        'img': img_path,
        'mask': gt_masks[i]
    }

### Setup utility functions

In [ ]:
def imshow(*imgs, title=None):
    fig_h = 8
    
    img_count = len(imgs)
    fig_w = 4 * img_count
    
    cmap = 'gray'
    
    fig = plt.figure(figsize=(fig_w, fig_h))
    if title is not None:
        fig.suptitle(title, fontsize=16, y=0.92)
    
    for i, img in enumerate(imgs):
        if img.dtype == np.bool:
            vmin = 0
            vmax = 1
            img = img.copy().astype(np.int)
        else:
            vmin = 0
            vmax = 255
        
        ax = plt.subplot(1, img_count, i + 1)
        plt.imshow(img, cmap=cmap, vmin=vmin, vmax=vmax)
        
        plt.axis('off')

def normalize_to_8bit(img):
    # normalize and convert to 8-bit
    img = img.copy()
    img = img - img.min()
    img = (img / img.max()) * 255
    img = np.floor(img).astype(np.uint8)
    
    return img

def create_mask_from_background(img, bkg_mask, std=3.0):
    # Sample the background, taking its mean/std value to use as a threshold
    img = img.copy()
    bkg_mean = np.mean(img[bkg_mask])
    print(bkg_mean)
    bkg_sdt = np.std(img[bkg_mask])

    img_tmp = np.zeros(img.shape)
    img_tmp[img > (bkg_mean + (std * bkg_sdt))] = 255
    img_mask = img_tmp.astype(np.uint8)
    
    return img_mask

def filter_contours_by_size(mask, min_size, max_size=None):
    ret, thresh = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(
        thresh,
        cv2.RETR_CCOMP,
        cv2.CHAIN_APPROX_SIMPLE
    )

    if max_size is None:
        max_size = mask.shape[0] * mask.shape[1]

    filtered_mask = thresh.copy()

    for i, c in enumerate(contours):
        # test the bounding area first, since it's quicker than rendering each contour
        rect = cv2.boundingRect(c)
        rect_area = rect[2] * rect[3]

        if rect_area > max_size or rect_area <= min_size:
            # Using a new blank mask to calculate actual size of each contour
            # because cv2.contourArea calculates the area using the Green
            # formula, which can give a different result than the number of
            # non-zero pixels.
            tmp_mask = np.zeros(mask.shape)
            cv2.drawContours(tmp_mask, contours, i, 1, cv2.FILLED, hierarchy=hierarchy)

            true_area = (tmp_mask > 0).sum()

            if true_area > max_size or true_area <= min_size:
                # Black-out contour pixels
                # TODO: Should probably collect the "bad" contours and erase them all at once
                cv2.drawContours(filtered_mask, contours, i, 0, cv2.FILLED)

    return filtered_mask

In [ ]:
imshow(bkg_mask, apt_ref_mask, title='reference masks')

# Pre & post processing

### Preprocessing

Preprocessing the apartment sub-regions may be helpful for different segmentation routines. 

First, the acquired images seem to have a very subtle undulating intensity variation seens as a 2-D grid in some thresholded masks. This may be the result of camera sensor irregularities or in the production of the chips themselves (slight bumps and valleys in the fabrication process).

The intensity variation is addressed by applying a slight blur to the apartment sub-region. Using a kernel size of 3 x 3 pixels achieves minimal alteration of image features while nearly eliminating the intensity variation. The preprocessing function below implements this option as a Boolean rather than a configurable kernel size to avoid complex interactions with the intermediate steps employed by different segmentation techniques.

The 2nd preprocessing option controls whether the parts of the image that are external to the apartment region are masked with the median value of the image. This removes the high-contrast apartment border that can cause ringing artifacts using various image processing methods such as blurring and convolution functions. 

NOTE: This implementation should likely change in production to choose the mask value from the median of a more reliable sub-region.

### Postprocessing

Postprocessing involves modifications to the mask resulting from a segmentation technique. This is currently limited to filtering contours by size to remove very small contours that some techniques are prone to generate and that might occur due to artifacts or debris in the image. And additional post-processing step could involve filling contour holes, but this is not yet implemented here.

In [ ]:
def preprocess_apt_img(input_img, apt_ref_mask, blur=True, median_mask=True):
    preproc_img = input_img.copy()
    
    if blur:
        preproc_img = cv2.blur(preproc_img, ksize=min_kern_size)
    if median_mask:
        apt_median = np.median(preproc_img)
        preproc_img[apt_ref_mask == False] = np.round(apt_median)
    
    return preproc_img

# Setup segmentation method functions

## Method #1: Difference of Gaussians (really a diff of blurs)

We perform a pseudo DoG...it's not really a difference of Gaussian's, 
but a difference of blurs.

The bilateral blur retains edge features while blurring the background, 
where the median blur is less selective. By subtracting the heavier median 
blur, the idea is to remove the noise while retaining the edge features. 

The blur images are cast to signed 16 bit to allow for negative 
values from the result of the subtraction.

**Inputs:**

 * median blur kernel size: should be >= bilateral kernel size
 * bilateral blur kernel size: ideally, should be <= size of feature of interest

## Method #2: Standard deviation convolution (matlab implementation)

https://www.mathworks.com/help/images/ref/stdfilt.html

## Method #3: Entropy convolution (skimage implementation)

https://scikit-image.org/docs/dev/auto_examples/filters/plot_entropy.html

## Method #4: Inverse Canny Edge (ICE)

This is similar to an inverse canny edge detection routine, though less sophisticated.

Basically, this is doing a minimal blur (3x3 kernel) to remove that grid artifact, then doing a mild bilateral filter to enhance the contrast of features. The next step is to perform a double threshold to get regions above and below the background sample. The last image shows combining the lower and upper thresholded regions.

What I like about this method is that it doesn't encroach the background sample region as much as the other methods, and is fairly simple and quick to calculate. The "magic" numbers for the kernels used are chosen as the lowest possible values to retain image detail.

## Method #5: 2-pass (Row/Column) rolling standard deviation

The method applies a rolling window function of the standard deviation along
each row of the image, and then again along each column. The resulting 2 images
are then thresholded using the background sample. The resulting masks are then
combined using a Boolean AND operation.

In [ ]:
# pseudo DoG method
def diff_of_blurs(
        input_img, 
        bkg_mask, 
        median_kernel_size=31, 
        bilat_kernel_size=7,
        bkg_stdev_thresh=3.0
):
    blur_median = cv2.medianBlur(input_img, ksize=median_kernel_size)
    blur_bilateral = cv2.bilateralFilter(input_img, d=bilat_kernel_size, sigmaColor=5, sigmaSpace=31)

    dog_img = blur_bilateral.astype(np.int16) - blur_median.astype(np.int16)

    # normalize and convert to 8-bit
    dog_img = normalize_to_8bit(dog_img)

    # mask result based on background sample
    dog_img_mask = create_mask_from_background(dog_img, bkg_mask, std=bkg_stdev_thresh)
    
    return dog_img, dog_img_mask

def std_dev_conv(
        input_img,
        bkg_mask,
        conv_kernel_size=11,
        bkg_stdev_thresh=3.0
):
    std_dev_kernel = np.ones((conv_kernel_size, conv_kernel_size))
    n = std_dev_kernel.sum()
    n1 = n - 1
    
    c1 = cv2.filter2D(
        input_img.astype(np.float32) ** 2, 
        -1, 
        std_dev_kernel / n1, 
        borderType=cv2.BORDER_REFLECT
    )
    c2 = cv2.filter2D(
        input_img.astype(np.float32), 
        -1, 
        std_dev_kernel, 
        borderType=cv2.BORDER_REFLECT
    )
    c2 = c2 ** 2 / (n * n1)
    
    std_dev_matlab_img = np.sqrt(np.maximum(c1 - c2, 0))

    # normalize and convert to 8-bit
    std_dev_matlab_img = normalize_to_8bit(std_dev_matlab_img)

    # mask result based on background sample
    std_dev_matlab_img_mask = create_mask_from_background(std_dev_matlab_img, bkg_mask, std=bkg_stdev_thresh)
    
    return std_dev_matlab_img, std_dev_matlab_img_mask

def entropy(
        input_img,
        bkg_mask,
        entropy_kernel_size=11,
        bkg_stdev_thresh=3.0
):
    ent_img = sk_entropy(input_img, disk(entropy_kernel_size))

    # normalize and convert to 8-bit
    ent_img = normalize_to_8bit(ent_img)
    
    # mask result based on background sample
    ent_mask = create_mask_from_background(ent_img, bkg_mask, std=bkg_stdev_thresh)
    
    return ent_img, ent_mask

def ice(
        input_img,
        bkg_mask,
        blur_kernel_size=5,
        bkg_stdev_thresh=3.0
):
    ice_img = cv2.bilateralFilter(~input_img, d=blur_kernel_size, sigmaColor=15, sigmaSpace=7)

    # normalize and convert to 8-bit
    ice_img = normalize_to_8bit(ice_img)

    # mask results based on background sample
    ice_mask1 = create_mask_from_background(ice_img, bkg_mask, std=bkg_stdev_thresh)
    ice_mask2 = create_mask_from_background(~ice_img, bkg_mask, std=bkg_stdev_thresh)
    
    ice_mask_final = np.logical_or(ice_mask1, ice_mask2).astype(np.uint8) * 255
    
    return ice_img, ice_mask_final

def rolling_2d_std_dev(
        input_img,
        roll_size=21,
        std_dev=3.0
):
    row_scan_img = np.zeros(input_img.shape, dtype=np.float64)
    col_scan_img = np.zeros(input_img.shape, dtype=np.float64)
    row_scan_mask = np.zeros(input_img.shape, dtype=np.bool)
    col_scan_mask = np.zeros(input_img.shape, dtype=np.bool)

    for i, row in enumerate(input_img):
        row_roll_std = pd.Series(row).rolling(roll_size, center=True).std()
        row_roll_std[row_roll_std.isna()] = 0
        row_scan_img[i, :] = row_roll_std
        row_scan_mask[i, :] = row_roll_std > std_dev

    for i, col in enumerate(input_img.T):
        col_roll_std = pd.Series(col).rolling(roll_size, center=True).std()
        col_roll_std[col_roll_std.isna()] = 0
        col_scan_img[:, i] = col_roll_std
        col_scan_mask[:, i] = col_roll_std > std_dev

    # create average from row/col images
    scan_img_combined = np.mean(np.array([row_scan_img, col_scan_img]), axis=0)
    # scan_img_combined = row_scan_img * col_scan_img
    scan_img_combined = normalize_to_8bit(scan_img_combined)
    
    # combine union of row/col masks
    scan_mask_combined = np.logical_and(row_scan_mask, col_scan_mask)
    scan_mask_combined = scan_mask_combined.astype(np.uint8) * 255

    # process result w/ morph close
    scan_mask_combo_close = cv2.morphologyEx(
        scan_mask_combined, 
        cv2.MORPH_CLOSE, 
        kernel=np.ones(min_kern_size)
    )
    
    return scan_img_combined, scan_mask_combo_close

# Example Runs

In [ ]:
# pick an apartment
# REMINDER: 
#     All data is loaded in the gt_data dictionary 
#     - keys are apt IDs,
#     - value is a dict with keys 'img' and 'mask' 
#     - 'img' & 'mask' hold the extracted apartment region and ground truth mask, respectively 
apt_id = '008_001'
apt_img = cv2.imread(gt_data[apt_id]['img'], cv2.IMREAD_GRAYSCALE)
gt_apt_mask = cv2.imread(gt_data[apt_id]['mask'], cv2.IMREAD_GRAYSCALE)

In [ ]:
imshow(apt_img, title=apt_id)

In [ ]:
def light_correction(input_img, ksize=15, sigma=3.0):
    img_blur = cv2.GaussianBlur(input_img, (ksize, ksize), sigma)
    img_corr = input_img / img_blur

    # Translate to zero, then normalize to 8-bit range
    img_corr = img_corr - img_corr.min()
    img_corr = np.floor((img_corr / img_corr.max()) * 255.0)
    img_corr = img_corr.astype(np.uint8)

    return img_corr

In [ ]:
# img_corr_base = light_correction(apt_img)
img_corr1 = light_correction(apt_img, sigma=3.0)
img_corr2 = light_correction(apt_img, ksize=15, sigma=5)
img_corr3 = light_correction(apt_img, ksize=21, sigma=5)
img_corr4 = light_correction(apt_img, ksize=7, sigma=5)

In [ ]:
imshow(apt_img, img_corr4, img_corr2, img_corr3)

### DoG

In [ ]:
median_kernel_size=31
bilat_kernel_size=7

dog_input = light_correction(apt_img, ksize=15, sigma=5.0)

# run the DoG method without any pre or post-processing
dog_img, dog_mask = diff_of_blurs(
    dog_input, bkg_mask, median_kernel_size=median_kernel_size, bilat_kernel_size=bilat_kernel_size
)

imshow(dog_img, dog_mask, title="%s - DoG - NoPre" % apt_id)

# run the DoG method with pre-processing of just blur
apt_img_preproc = preprocess_apt_img(dog_input, apt_ref_mask, blur=True, median_mask=False)
dog_img, dog_mask = diff_of_blurs(
    apt_img_preproc, bkg_mask, median_kernel_size=median_kernel_size, bilat_kernel_size=bilat_kernel_size
)
imshow(dog_img, dog_mask, title="%s - DoG - PreBlurOnly" % apt_id)

# run the DoG method with pre-processing of min blur and external apt median masking
apt_img_preproc = preprocess_apt_img(dog_input, apt_ref_mask, blur=True, median_mask=True)
dog_img, dog_mask = diff_of_blurs(
    apt_img_preproc, bkg_mask, median_kernel_size=median_kernel_size, bilat_kernel_size=bilat_kernel_size
)
imshow(dog_img, dog_mask, title="%s - DoG - Pre blur+mask" % apt_id)

# Run Dog with all pre-processing and post-processing
# run the DoG method with pre-processing of min blur and external apt median masking
apt_img_preproc = preprocess_apt_img(apt_img, apt_ref_mask, blur=True, median_mask=True)
dog_img, dog_mask = diff_of_blurs(
    apt_img_preproc, bkg_mask, median_kernel_size=median_kernel_size, bilat_kernel_size=bilat_kernel_size
)
dog_mask = filter_contours_by_size(dog_mask, min_size=9 * 9)

imshow(dog_img, dog_mask, title="%s - DoG - FullPre + Post" % apt_id)

### StdDev Conv

In [ ]:
conv_kern_size = 5
std_dev = 12.0

# run the DoG method without any pre or post-processing
output_img, output_mask = std_dev_conv(apt_img, bkg_mask, conv_kernel_size=conv_kern_size)

imshow(output_img, output_mask, title="%s - Std - NoPre" % apt_id)

# run the DoG method with pre-processing of just blur
apt_img_preproc = preprocess_apt_img(apt_img, apt_ref_mask, blur=True, median_mask=False)
output_img, output_mask = std_dev_conv(
    apt_img_preproc, bkg_mask, conv_kernel_size=conv_kern_size, bkg_stdev_thresh=std_dev
)

imshow(output_img, output_mask, title="%s - Std - PreBlurOnly" % apt_id)

# run the DoG method with pre-processing of min blur and external apt median masking
apt_img_preproc = preprocess_apt_img(apt_img, apt_ref_mask, blur=True, median_mask=True)
output_img, output_mask = std_dev_conv(
    apt_img_preproc, bkg_mask, conv_kernel_size=conv_kern_size, bkg_stdev_thresh=std_dev
)
imshow(output_img, output_mask, title="%s - Std - Pre blur+mask" % apt_id)

# Run Dog with all pre-processing and post-processing
# run the DoG method with pre-processing of min blur and external apt median masking
apt_img_preproc = preprocess_apt_img(apt_img, apt_ref_mask, blur=True, median_mask=True)
output_img, output_mask = std_dev_conv(
    apt_img_preproc, bkg_mask, conv_kernel_size=conv_kern_size, bkg_stdev_thresh=std_dev
)
output_mask = filter_contours_by_size(output_mask, min_size=9 * 9)

imshow(output_img, output_mask, title="%s - Std - FullPre + Post" % apt_id)

### Entropy

In [ ]:
ent_kern_size = 5
std_dev = 6.0

# run method without any pre or post-processing
output_img, output_mask = entropy(
    apt_img, bkg_mask, entropy_kernel_size=ent_kern_size, bkg_stdev_thresh=std_dev
)

imshow(output_img, output_mask, title="%s - Ent - NoPre" % apt_id)

# run with pre-processing of just blur
apt_img_preproc = preprocess_apt_img(apt_img, apt_ref_mask, blur=True, median_mask=False)
output_img, output_mask = entropy(
    apt_img_preproc, bkg_mask, entropy_kernel_size=ent_kern_size, bkg_stdev_thresh=std_dev
)

imshow(output_img, output_mask, title="%s - Ent - PreBlurOnly" % apt_id)

# run with pre-processing of min blur and external apt median masking
apt_img_preproc = preprocess_apt_img(apt_img, apt_ref_mask, blur=True, median_mask=True)
output_img, output_mask = entropy(
    apt_img_preproc, bkg_mask, entropy_kernel_size=ent_kern_size, bkg_stdev_thresh=std_dev
)
imshow(output_img, output_mask, title="%s - Ent - Pre blur+mask" % apt_id)

# run with all pre-processing and post-processing
apt_img_preproc = preprocess_apt_img(apt_img, apt_ref_mask, blur=True, median_mask=True)
output_img, output_mask = entropy(
    apt_img_preproc, bkg_mask, entropy_kernel_size=ent_kern_size, bkg_stdev_thresh=std_dev
)
output_mask = filter_contours_by_size(output_mask, min_size=9 * 9)

imshow(output_img, output_mask, title="%s - Ent - FullPre + Post" % apt_id)

### ICE

In [ ]:
ice_kern_size = 13
std_dev = 6.0

meth_input = apt_img.copy()
meth_input = light_correction(dog_input, ksize=31, sigma=3.0)

# run method without any pre or post-processing
output_img, output_mask = ice(meth_input, bkg_mask, blur_kernel_size=ice_kern_size, bkg_stdev_thresh=std_dev)

imshow(output_img, output_mask, title="%s - DoG - NoPre" % apt_id)

# run with pre-processing of just blur
apt_img_preproc = preprocess_apt_img(meth_input, apt_ref_mask, blur=True, median_mask=False)
output_img, output_mask = ice(apt_img_preproc, bkg_mask, blur_kernel_size=ice_kern_size, bkg_stdev_thresh=std_dev)

imshow(output_img, output_mask, title="%s - DoG - PreBlurOnly" % apt_id)

# run with pre-processing of min blur and external apt median masking
apt_img_preproc = preprocess_apt_img(meth_input, apt_ref_mask, blur=True, median_mask=True)
output_img, output_mask = ice(apt_img_preproc, bkg_mask, blur_kernel_size=ice_kern_size, bkg_stdev_thresh=std_dev)

imshow(output_img, output_mask, title="%s - DoG - Pre blur+mask" % apt_id)

# run with all pre-processing and post-processing
apt_img_preproc = preprocess_apt_img(meth_input, apt_ref_mask, blur=True, median_mask=True)
output_img, output_mask = ice(apt_img_preproc, bkg_mask, blur_kernel_size=ice_kern_size, bkg_stdev_thresh=std_dev)

output_mask = filter_contours_by_size(output_mask, min_size=9 * 9)

imshow(output_img, output_mask, title="%s - DoG - FullPre + Post" % apt_id)

### Rolling 2-D Std Dev

In [ ]:
roll_size = 21
std_dev = 2.0

meth_input = apt_img.copy()
meth_input = light_correction(dog_input, ksize=31, sigma=3.0)

# run method without any pre or post-processing
output_img, output_mask = rolling_2d_std_dev(meth_input, roll_size=roll_size, std_dev=std_dev)

imshow(output_img, output_mask, title="%s - DoG - NoPre" % apt_id)

# run with pre-processing of just blur
apt_img_preproc = preprocess_apt_img(meth_input, apt_ref_mask, blur=True, median_mask=False)
output_img, output_mask = rolling_2d_std_dev(apt_img_preproc, roll_size=roll_size, std_dev=std_dev)

imshow(output_img, output_mask, title="%s - DoG - PreBlurOnly" % apt_id)

# run with pre-processing of min blur and external apt median masking
apt_img_preproc = preprocess_apt_img(meth_input, apt_ref_mask, blur=True, median_mask=True)
output_img, output_mask = rolling_2d_std_dev(apt_img_preproc, roll_size=roll_size, std_dev=std_dev)

imshow(output_img, output_mask, title="%s - DoG - Pre blur+mask" % apt_id)

# run with all pre-processing and post-processing
apt_img_preproc = preprocess_apt_img(meth_input, apt_ref_mask, blur=True, median_mask=True)
output_img, output_mask = rolling_2d_std_dev(apt_img_preproc, roll_size=roll_size, std_dev=std_dev)

output_mask = filter_contours_by_size(output_mask, min_size=9 * 9)

imshow(output_img, output_mask, title="%s - DoG - FullPre + Post" % apt_id)